In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union

def load_moex_data(security='GAZP', years=5):
    """
    Загружает исторические данные с MOEX для указанной ценной бумаги.
    """
    # Формируем даты начала и конца периода
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=365 * years)
    
    # Базовый URL ISS API для исторических данных
    base_url = "https://iss.moex.com/iss/history/engines/stock/markets/shares/securities"
    
    # Параметры запроса
    params = {
        'from': start_date.strftime('%Y-%m-%d'),
        'till': end_date.strftime('%Y-%m-%d'),
        'start': 0
    }
    
    # Список для накопления данных
    all_data = []
    
    while True:
        # Формируем URL для запроса
        url = f"{base_url}/{security}.json"
        
        # Делаем запрос к API
        try:
            r = requests.get(url, params=params)
            r.raise_for_status()  # Проверяем на ошибки
            data = r.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при загрузке данных для {security}: {e}")
            return None
        
        # Получаем данные из секции 'history'
        history_data = data.get('history', {}).get('data', [])
        if not history_data:
            break  # Нет данных или достигнут конец
        
        all_data.extend(history_data)
        
        # Проверяем, есть ли еще данные
        cursor = data.get('history.cursor', {}).get('data', [])
        if not cursor:
            break
        
        # Получаем информацию о пагинации
        total_rows = cursor[0][1]
        start = cursor[0][0] + cursor[0][2]  # current_start + page_size
        
        if start >= total_rows:
            break
            
        params['start'] = start
    
    # Создаем DataFrame
    columns = data.get('history', {}).get('columns', [])
    df = pd.DataFrame(all_data, columns=columns)
    
    # Фильтруем только по основному режиму торгов TQBR
    df = df[df['BOARDID'] == 'TQBR']
    
    # Оставляем только нужные колонки
    needed_columns = [
        'TRADEDATE',
        'SECID',
        'OPEN',
        'HIGH',
        'LOW',
        'CLOSE',
        'VOLUME',
        'WAPRICE'
    ]
    df = df[needed_columns]
    
    # Преобразуем даты и сортируем
    df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
    df = df.sort_values('TRADEDATE').reset_index(drop=True)
    
    return df

def calculate_market_stress_index(securities: Dict[str, str] = {
        'IMOEX': 'market_index',
        'RTSI': 'rts_index',
        'RVI': 'volatility_index',
        'USDRUS': 'usdrub_rate',
        'RUGBITR': 'govt_bond_index'
    }, 
    lookback_period: int = 60) -> pd.DataFrame:
    """
    Рассчитывает упрощенный индекс рыночного стресса на основе доступных данных MOEX.
    """
    all_data = {}
    
    # Загружаем данные по каждому инструменту
    for ticker in securities.keys():
        try:
            df = load_moex_data(security=ticker, years=2)
            if df is not None and not df.empty:
                all_data[ticker] = df
            else:
                print(f"Нет данных для {ticker}")
        except Exception as e:
            print(f"Ошибка при загрузке {ticker}: {e}")
    
    if not all_data:
        raise ValueError("Не удалось загрузить данные ни по одному инструменту")
    
    # Создаем DataFrame с общими датами
    result = pd.DataFrame()
    base_dates = None
    
    # Находим общие даты для всех инструментов
    for ticker, df in all_data.items():
        if base_dates is None:
            base_dates = set(df['TRADEDATE'])
        else:
            base_dates = base_dates.intersection(set(df['TRADEDATE']))
    
    base_dates = sorted(list(base_dates))
    result['TRADEDATE'] = base_dates
    
    # 1. Волатильность рынка (нормализованная)
    if 'IMOEX' in all_data:
        df = all_data['IMOEX']
        df = df[df['TRADEDATE'].isin(base_dates)]
        returns = df['CLOSE'].pct_change()
        volatility = returns.rolling(window=lookback_period).std() * np.sqrt(252)
        result['Market_Volatility'] = (volatility - volatility.mean()) / volatility.std()
    
    # 2. Волатильность валютного рынка
    if 'USDRUS' in all_data:
        df = all_data['USDRUS']
        df = df[df['TRADEDATE'].isin(base_dates)]
        fx_returns = df['CLOSE'].pct_change()
        fx_volatility = fx_returns.rolling(window=lookback_period).std() * np.sqrt(252)
        result['FX_Volatility'] = (fx_volatility - fx_volatility.mean()) / fx_volatility.std()
    
    # 3. Индекс волатильности RVI
    if 'RVI' in all_data:
        df = all_data['RVI']
        df = df[df['TRADEDATE'].isin(base_dates)]
        result['RVI_Normalized'] = (df['CLOSE'] - df['CLOSE'].mean()) / df['CLOSE'].std()
    
    # 4. Аномальные объемы торгов
    if 'IMOEX' in all_data:
        df = all_data['IMOEX']
        df = df[df['TRADEDATE'].isin(base_dates)]
        volume_ma = df['VOLUME'].rolling(window=lookback_period).mean()
        volume_std = df['VOLUME'].rolling(window=lookback_period).std()
        volume_z_score = (df['VOLUME'] - volume_ma) / volume_std
        result['Volume_Stress'] = volume_z_score
    
    # 5. Корреляция между индексами
    if 'IMOEX' in all_data and 'RTSI' in all_data:
        def rolling_correlation(x, y, window):
            return pd.Series(x).rolling(window).corr(pd.Series(y))
        
        imoex_df = all_data['IMOEX']
        rtsi_df = all_data['RTSI']
        imoex_df = imoex_df[imoex_df['TRADEDATE'].isin(base_dates)]
        rtsi_df = rtsi_df[rtsi_df['TRADEDATE'].isin(base_dates)]
        
        corr = rolling_correlation(imoex_df['CLOSE'].pct_change(),
                                 rtsi_df['CLOSE'].pct_change(),
                                 lookback_period)
        result['Index_Correlation'] = (corr - corr.mean()) / corr.std()
    
    # Расчет итогового индекса стресса
    stress_components = [col for col in result.columns if col != 'TRADEDATE']
    if stress_components:
        result['Stress_Index'] = result[stress_components].mean(axis=1)
        
        # Нормализуем итоговый индекс
        result['Stress_Index'] = (result['Stress_Index'] - result['Stress_Index'].mean()) / \
                                result['Stress_Index'].std()
        
        # Масштабируем к диапазону официального индекса стресса (0-5)
        result['Stress_Index'] = 2.5 + result['Stress_Index']
        
        # Добавляем категории стресса
        result['Stress_Category'] = pd.cut(result['Stress_Index'],
                                         bins=[-np.inf, 1, 2, 2.5, 3, np.inf],
                                         labels=['Низкий', 'Умеренный', 'Повышенный', 
                                               'Высокий', 'Кризисный'])
    
    return result

def plot_stress_index(stress_df: pd.DataFrame) -> None:
    """
    Визуализирует индекс стресса и его компоненты
    """
    import matplotlib.pyplot as plt
    
    # Создаем график
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # График индекса стресса
    stress_df.plot(x='TRADEDATE', y='Stress_Index', ax=ax1, color='red', label='Индекс стресса')
    ax1.axhline(y=2.5, color='yellow', linestyle='--', label='Порог кризиса')
    ax1.set_title('Индекс рыночного стресса')
    ax1.legend()
    ax1.grid(True)
    
    # График компонент
    components = [col for col in stress_df.columns 
                 if col not in ['TRADEDATE', 'Stress_Index', 'Stress_Category']]
    stress_df.plot(x='TRADEDATE', y=components, ax=ax2)
    ax2.set_title('Компоненты индекса стресса')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Рассчитываем индекс стресса
    try:
        stress_df = calculate_market_stress_index()
        
        # Выводим текущее значение
        current_stress = stress_df['Stress_Index'].iloc[-1]
        current_category = stress_df['Stress_Category'].iloc[-1]
        print(f"Текущий уровень стресса: {current_stress:.2f} ({current_category})")
        
        # Визуализируем результаты
        plot_stress_index(stress_df)
    except Exception as e:
        print(f"Ошибка при расчете индекса стресса: {e}")